In [1]:
import requests
import sys
import pandas as pd
import urllib.request
import re
import os
import glob
import gzip
from ftplib import FTP
from io import StringIO
from Bio import SeqIO
from tqdm import tqdm

In [12]:
url = 'http://www.ensembl.org/biomart/martservice?type=datasets&mart=ENSEMBL_MART_ENSEMBL'
response = requests.get(url).text
df = pd.read_csv(StringIO(response), delimiter='\t', header=None)
datasets = df[1].tolist()

In [2]:
species = ['acanthochromis_polyacanthus', 'accipiter_nisus', 'ailuropoda_melanoleuca', 'amazona_collaria', 'amphilophus_citrinellus', 'amphiprion_ocellaris', 'amphiprion_percula', 'anabas_testudineus', 'anas_platyrhynchos', 'anas_platyrhynchos_platyrhynchos', 'anas_zonorhyncha', 'ancestral_alleles', 'anolis_carolinensis', 'anser_brachyrhynchus', 'anser_cygnoides', 'aotus_nancymaae', 'apteryx_haastii', 'apteryx_owenii', 'apteryx_rowi', 'aquila_chrysaetos_chrysaetos', 'astatotilapia_calliptera', 'astyanax_mexicanus', 'astyanax_mexicanus_pachon', 'athene_cunicularia', 'balaenoptera_musculus', 'betta_splendens', 'bison_bison_bison', 'bos_grunniens', 'bos_indicus_hybrid', 'bos_mutus', 'bos_taurus', 'bos_taurus_hybrid', 'bubo_bubo', 'buteo_japonicus', 'caenorhabditis_elegans', 'cairina_moschata_domestica', 'calidris_pugnax', 'calidris_pygmaea', 'callithrix_jacchus', 'callorhinchus_milii', 'camarhynchus_parvulus', 'camelus_dromedarius', 'canis_lupus_dingo', 'canis_lupus_familiaris', 'canis_lupus_familiarisbasenji', 'canis_lupus_familiarisboxer', 'canis_lupus_familiarisgreatdane', 'canis_lupus_familiarisgsd', 'capra_hircus', 'capra_hircus_blackbengal', 'carassius_auratus', 'carlito_syrichta', 'castor_canadensis', 'catagonus_wagneri', 'catharus_ustulatus', 'cavia_aperea', 'cavia_porcellus', 'cebus_imitator', 'cercocebus_atys', 'cervus_hanglu_yarkandensis', 'chelonoidis_abingdonii', 'chelydra_serpentina', 'chinchilla_lanigera', 'chlorocebus_sabaeus', 'choloepus_hoffmanni', 'chrysemys_picta_bellii', 'chrysolophus_pictus', 'ciona_intestinalis', 'ciona_savignyi', 'clupea_harengus', 'colobus_angolensis_palliatus', 'corvus_moneduloides', 'cottoperca_gobio', 'coturnix_japonica', 'cricetulus_griseus_chok1gshd', 'cricetulus_griseus_crigri', 'cricetulus_griseus_picr', 'crocodylus_porosus', 'cyanistes_caeruleus', 'cyclopterus_lumpus', 'cynoglossus_semilaevis', 'cyprinodon_variegatus', 'cyprinus_carpio', 'cyprinus_carpio_carpio', 'cyprinus_carpio_germanmirror', 'cyprinus_carpio_hebaored', 'cyprinus_carpio_huanghe', 'danio_rerio', 'dasypus_novemcinctus', 'delphinapterus_leucas', 'denticeps_clupeoides', 'dicentrarchus_labrax', 'dipodomys_ordii', 'dromaius_novaehollandiae', 'drosophila_melanogaster', 'echeneis_naucrates', 'echinops_telfairi', 'electrophorus_electricus', 'eptatretus_burgeri', 'equus_asinus', 'equus_asinus_asinus', 'equus_caballus', 'erinaceus_europaeus', 'erpetoichthys_calabaricus', 'erythrura_gouldiae', 'esox_lucius', 'falco_tinnunculus', 'felis_catus', 'ficedula_albicollis', 'fukomys_damarensis', 'fundulus_heteroclitus', 'gadus_morhua', 'gallus_gallus', 'gallus_gallus_gca000002315v5', 'gallus_gallus_gca016700215v2', 'gambusia_affinis', 'gasterosteus_aculeatus', 'geospiza_fortis', 'gopherus_agassizii', 'gopherus_evgoodei', 'gorilla_gorilla', 'gouania_willdenowi', 'haplochromis_burtoni', 'heterocephalus_glaber_female', 'heterocephalus_glaber_male', 'hippocampus_comes', 'homo_sapiens', 'hucho_hucho', 'ictalurus_punctatus', 'ictidomys_tridecemlineatus', 'jaculus_jaculus', 'junco_hyemalis', 'kryptolebias_marmoratus', 'labrus_bergylta', 'larimichthys_crocea', 'lates_calcarifer', 'laticauda_laticaudata', 'latimeria_chalumnae', 'lepidothrix_coronata', 'lepisosteus_oculatus', 'leptobrachium_leishanense', 'lonchura_striata_domestica', 'loxodonta_africana', 'lynx_canadensis', 'macaca_fascicularis', 'macaca_mulatta', 'macaca_nemestrina', 'malurus_cyaneus_samueli', 'manacus_vitellinus', 'mandrillus_leucophaeus', 'marmota_marmota_marmota', 'mastacembelus_armatus', 'maylandia_zebra', 'meleagris_gallopavo', 'melopsittacus_undulatus', 'meriones_unguiculatus', 'mesocricetus_auratus', 'microcebus_murinus', 'microtus_ochrogaster', 'mola_mola', 'monodelphis_domestica', 'monodon_monoceros', 'monopterus_albus', 'moschus_moschiferus', 'mus_caroli', 'mus_musculus', 'mus_musculus_129s1svimj', 'mus_musculus_aj', 'mus_musculus_akrj', 'mus_musculus_balbcj', 'mus_musculus_c3hhej', 'mus_musculus_c57bl6nj', 'mus_musculus_casteij', 'mus_musculus_cbaj', 'mus_musculus_dba2j', 'mus_musculus_fvbnj', 'mus_musculus_lpj', 'mus_musculus_nodshiltj', 'mus_musculus_nzohlltj', 'mus_musculus_pwkphj', 'mus_musculus_wsbeij', 'mus_pahari', 'mus_spicilegus', 'mus_spretus', 'mustela_putorius_furo', 'myotis_lucifugus', 'myripristis_murdjan', 'naja_naja', 'nannospalax_galili', 'neogobius_melanostomus', 'neolamprologus_brichardi', 'neovison_vison', 'nomascus_leucogenys', 'notamacropus_eugenii', 'notechis_scutatus', 'nothobranchius_furzeri', 'nothoprocta_perdicaria', 'numida_meleagris', 'ochotona_princeps', 'octodon_degus', 'oncorhynchus_kisutch', 'oncorhynchus_mykiss', 'oncorhynchus_tshawytscha', 'oreochromis_aureus', 'oreochromis_niloticus', 'ornithorhynchus_anatinus', 'oryctolagus_cuniculus', 'oryzias_javanicus', 'oryzias_latipes', 'oryzias_latipes_hni', 'oryzias_latipes_hsok', 'oryzias_melastigma', 'oryzias_sinensis', 'otolemur_garnettii', 'otus_sunia', 'ovis_aries', 'ovis_aries_rambouillet', 'pan_paniscus', 'pan_troglodytes', 'panthera_leo', 'panthera_pardus', 'panthera_tigris_altaica', 'papio_anubis', 'parambassis_ranga', 'paramormyrops_kingsleyae', 'parus_major', 'pavo_cristatus', 'pelodiscus_sinensis', 'pelusios_castaneus', 'periophthalmus_magnuspinnatus', 'peromyscus_maniculatus_bairdii', 'petromyzon_marinus', 'phascolarctos_cinereus', 'phasianus_colchicus', 'phocoena_sinus', 'physeter_catodon', 'piliocolobus_tephrosceles', 'podarcis_muralis', 'poecilia_formosa', 'poecilia_latipinna', 'poecilia_mexicana', 'poecilia_reticulata', 'pogona_vitticeps', 'pongo_abelii', 'procavia_capensis', 'prolemur_simus', 'propithecus_coquereli', 'pseudonaja_textilis', 'pteropus_vampyrus', 'pundamilia_nyererei', 'pygocentrus_nattereri', 'rattus_norvegicus', 'rhinolophus_ferrumequinum', 'rhinopithecus_bieti', 'rhinopithecus_roxellana', 'saccharomyces_cerevisiae', 'saimiri_boliviensis_boliviensis', 'salarias_fasciatus', 'salmo_salar', 'salmo_trutta', 'salvator_merianae', 'sander_lucioperca', 'sarcophilus_harrisii', 'sciurus_vulgaris', 'scleropages_formosus', 'scophthalmus_maximus', 'serinus_canaria', 'seriola_dumerili', 'seriola_lalandi_dorsalis', 'sinocyclocheilus_anshuiensis', 'sinocyclocheilus_grahami', 'sinocyclocheilus_rhinocerous', 'sorex_araneus', 'sparus_aurata', 'spermophilus_dauricus', 'sphaeramia_orbicularis', 'sphenodon_punctatus', 'stachyris_ruficeps', 'stegastes_partitus', 'strigops_habroptila', 'strix_occidentalis_caurina', 'struthio_camelus_australis', 'suricata_suricatta', 'sus_scrofa', 'sus_scrofa_bamei', 'sus_scrofa_berkshire', 'sus_scrofa_hampshire', 'sus_scrofa_jinhua', 'sus_scrofa_landrace', 'sus_scrofa_largewhite', 'sus_scrofa_meishan', 'sus_scrofa_pietrain', 'sus_scrofa_rongchang', 'sus_scrofa_tibetan', 'sus_scrofa_usmarc', 'sus_scrofa_wuzhishan', 'taeniopygia_guttata', 'takifugu_rubripes', 'terrapene_carolina_triunguis', 'tetraodon_nigroviridis', 'theropithecus_gelada', 'tupaia_belangeri', 'tursiops_truncatus', 'urocitellus_parryii', 'ursus_americanus', 'ursus_maritimus', 'ursus_thibetanus_thibetanus', 'varanus_komodoensis', 'vicugna_pacos', 'vombatus_ursinus', 'vulpes_vulpes', 'xenopus_tropicalis', 'xiphophorus_couchianus', 'xiphophorus_maculatus', 'zalophus_californianus', 'zonotrichia_albicollis', 'zosterops_lateralis_melanops']
print(len(species))

317


In [3]:
base_url = 'ftp.ensembl.org'
base_path = '/pub/release-109/fasta'

# Connect to FTP server
ftp = FTP(base_url)
ftp.login()

# Iterate over each species
for specie in tqdm(species):
    try:
        species_path = f"{base_path}/{specie}/cds/"

        # Change directory to 'cds' folder
        ftp.cwd(species_path)

        # Get the list of files in the 'cds' folder
        file_list = ftp.nlst()

        # Download files ending with '.fa.gz'
        for file_name in file_list:
            if file_name.endswith('.fa.gz'):
                local_file_path = os.path.join(specie, 'cds', file_name)
                local_dir = os.path.dirname(local_file_path)

                # Create local directories if they don't exist
                os.makedirs(local_dir, exist_ok=True)

                # Download the file
                with open(local_file_path, 'wb') as file:
                    ftp.retrbinary(f'RETR {file_name}', file.write)
    except:
        continue

# Close the FTP connection
ftp.quit()


100%|████████████████████████████████████████████████████████████████████████████████| 317/317 [18:51<00:00,  3.57s/it]


'221 Goodbye.'

In [4]:
# Directory containing .gz files
directory = './cds_data/'

# Output file to store the merged content
output_file = 'merged.txt'

# Get the list of .gz files in the directory
gz_files = glob.glob(f"{directory}/**/*.gz", recursive=True)

with open(output_file, 'w') as output:
    for gz_file in tqdm(gz_files):
        with gzip.open(gz_file, 'rt') as file:
            content = file.read()
            output.write(content)

100%|████████████████████████████████████████████████████████████████████████████████| 315/315 [05:20<00:00,  1.02s/it]

Extraction and merging complete!
